# Assignment Prompt and Generation


This notebook demonstrates how to generate unique assignment prompts and sample student essays using the OpenAI API. The setup includes essential libraries, environment configuration, prompt generation, and essay generation with randomized quality levels.

This notebook generates assignment prompts and sample student essays for specified grade levels and subjects using the OpenAI API.

### Outline:
1. **Setup**
   - Import required libraries.
   - Load environment variables for API access.
   - Define quality levels for essay grading.

2. **Functions**
   - **generate_prompts**: Generates 10 assignment prompts for each grade-subject pair.
   - **generate_sample_essay**: Creates a sample essay for each prompt with a randomly assigned quality level.

3. **Execution**
   - Define grade-subject pairs.
   - Generate and display prompts.
   - Generate and display sample essays with random quality levels.
   
### Requirements:
- **API Key**: Ensure that `OPENAI_API_KEY` is set in the `.env` file in the project directory.
- **Installed Libraries**:
  - `openai`
  - `IPython.display`
  - `python-dotenv`
  - `re` (comes with Python)
  - `random` (comes with Python)








In [1]:
# Import necessary libraries
import openai  # OpenAI API for generating text based on prompts
from IPython.display import display, Markdown  # Markdown display tools for Jupyter Notebooks
import os  # OS library for file and directory interactions
from dotenv import load_dotenv, find_dotenv  # dotenv utilities for managing environment variables
import re  # Regular expressions library for text pattern matching and formatting
import random  # Random module for generating randomized values, such as quality levels

# Function to convert text to Markdown format, allowing formatted display in Jupyter Notebooks
def to_markdown(text):
    # Wraps text in Markdown format for enhanced readability and presentation in Jupyter Notebooks
    return Markdown(text)


In [2]:
# Automatically locate and load the .env file with environment variables
env_path = find_dotenv()  # Searches for the .env file in the project directory
load_dotenv(dotenv_path=env_path)  # Loads the environment variables from .env

# Set the OpenAI API key from environment variable (pulled from the .env file)
openai.api_key = os.getenv('OPENAI_API_KEY')


### Function: `generate_prompts`

#### Description:
- This function generates a list of assignment prompts tailored to specific grade-subject pairs using the OpenAI API.
- The function takes a list of grade-subject pairs, constructs a unique prompt for each, and requests 10 age-appropriate assignment prompts for each pair.
- The prompts are displayed in Markdown format within Jupyter Notebooks for easy viewing.

#### Parameters:
- **pairs**: A list of tuples, where each tuple consists of:
  - **grade_level**: A string representing the grade level (e.g., "11" for 11th grade).
  - **subject**: A string representing the academic subject for which prompts are being generated (e.g., "US History").

#### Process:
1. **Prompt Construction**: For each grade-subject pair, the function builds a specific request prompt that asks the OpenAI model to generate 10 unique, challenging, and engaging assignment prompts appropriate for that grade level and subject.
2. **API Call**: The function sends this constructed prompt to the OpenAI API (`gpt-3.5-turbo`), instructing it to generate relevant assignment ideas.
3. **Formatting and Cleaning**: After receiving the response:
   - Each prompt is stripped of any leading numbering or formatting inconsistencies using regular expressions to ensure a clean display.
   - Only the first 10 prompts are kept, even if more are returned.
4. **Display**: Each prompt is then labeled (e.g., "Prompt 1," "Prompt 2") and displayed in Markdown format for readability.

#### Output:
- **Returns**: A dictionary (`all_prompts`) where:
  - Each key is a tuple of the form `(grade_level, subject)`.
  - Each value is a list of 10 formatted assignment prompts for the corresponding grade-subject pair.



In [3]:
# Function to generate assignment prompts based on a list of grade-subject pairs
def generate_prompts(pairs):
    all_prompts = {}
    for grade_level, subject in pairs:
        prompt_text = f"""
        Role: You are a {grade_level}th-grade {subject} teacher. Please create 10 unique assignment prompts.

        Each prompt should:
        - Be age-appropriate for {grade_level}th grade
        - Challenge students' understanding, encouraging critical thinking and engagement with the subject matter
        - Cover diverse themes or topics within {subject}
        
        Ensure each prompt is clear, concise, and engaging.
        """
        
        # Generate prompts using OpenAI API
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a creative and insightful prompt generator for teachers."},
                {"role": "user", "content": prompt_text}
            ]
        )
        
        # Extract prompts and clean up formatting
        prompts_text = completion.choices[0].message['content']
        prompts_list = [re.sub(r'^\d+\.\s*', '', prompt.strip()) for prompt in prompts_text.split('\n') if prompt.strip()]
        prompts_list = prompts_list[:10]  # Keep exactly 10 prompts
        all_prompts[(grade_level, subject)] = prompts_list
        
        # Display prompts in Markdown format
        display(Markdown(f"### Generated Prompts for {grade_level}th Grade {subject}"))
        for i, prompt in enumerate(prompts_list, start=1):
            display(Markdown(f"**Prompt {i}:** {prompt}"))
    
    return all_prompts


### Function: `generate_sample_essay`

#### Description:
- This function generates a sample essay for each grade-subject pair based on a specific prompt and a randomly assigned quality level. 
- It leverages a previously generated list of prompts, selects a specified prompt, and instructs OpenAI to generate a sample essay typical of a student at that grade level and subject.

#### Parameters:
- **all_prompts**: A dictionary where each key is a tuple `(grade_level, subject)` and each value is a list of 10 assignment prompts generated for that grade-subject pair.
- **pairs**: A list of grade-subject tuples, where each tuple consists of:
  - **grade_level**: A string indicating the grade level (e.g., "11" for 11th grade).
  - **subject**: A string representing the academic subject (e.g., "US History").
- **prompt_index**: An integer indicating which prompt from the list to use for essay generation (default is 0, which selects the first prompt in the list).

#### Process:
1. **Prompt Selection**: For each grade-subject pair in `pairs`, the function retrieves the specified prompt from `all_prompts` using `prompt_index`.
2. **Quality Level Assignment**: A quality level (e.g., "A", "B", "C", "D", or "F") is randomly selected for the essay using `random.choice(quality_levels)`.
3. **Essay Generation Prompt**: An essay prompt is constructed based on the selected prompt and assigned quality level, simulating a student essay typical of that grade and subject.
4. **API Call**: The function sends the essay prompt to the OpenAI API (`gpt-3.5-turbo`), requesting a sample response.
5. **Error Handling**: If prompts are missing or if `prompt_index` is out of range, an error message is printed. Any API errors are caught and logged, and the essay output for that pair is set to `None`.
6. **Display and Storage**: Each generated essay, along with its quality level, is displayed in Markdown format and stored in a dictionary for further use.

#### Output:
- **Returns**: A dictionary (`all_essays`) where:
  - Each key is a tuple `(grade_level, subject)`.
  - Each value is a dictionary with:
    - **quality**: The randomly assigned quality level for the essay.
    - **essay**: The generated essay text or `None` if an error occurred.

#### Example Usage:
```python
grade_subject_pairs = [("11", "US History"), ("10", "Global History")]
all_prompts = generate_prompts(grade_subject_pairs)
all_essays = generate_sample_essay(all_prompts, grade_subject_pairs, prompt_index=0)


In [4]:
# Want a mean grade of satisfactory, so we make a mean distribution around satisfactor
# For each category, sample from that distribution and get a grade and a number of points for that particular criteria
# We will get different grades for each criteria
# Another important part is we need to treat criteria as independent units (unit for fine tuning) rather than grading-rubrics
# We need to get the criteria correct and we can think of a grading rubric as a combination of criteria 
# We need an LLM to transform what it found in the essays and produce a final grade for the criteria and grading rubric
# If we break down by criteria, we need our final data to be organized: final essay is input, criteria is unit of analyis, and output will be for each criteria, we need an analysis 
# For grammar, should our model be able to idenitfy all the grammar mistakes? Is grammar mistakes enough? Should it identify good uses of grammar? Correclty using grammar 
# Then we need additional level of summarization to the student and the student needs

# Identify 4 or 5 criteria that are simple but relevant to our task 
# We need to find ways to find a set of mutually exclusive criteria and create a full table of all of them
# We need to think of how to evaluate different LLMs and what is the perfect response, and then we use the perfect response to fine-tune and then use that evaluation method to select that LLM
# Still use the rubric to create the essay, but now we will have one output per essay per criteria 
# After we start with one criteria, we perfect, 


def generate_multiple_essays_with_selected_qualities(prompt, grade_level, subject, qualities):
    essays_with_explanations = []
    
    for quality in qualities:
        # Primary prompt to generate the essay
        essay_prompt = f"""
        You are a {grade_level}th-grade student in {subject} with a {quality} average. Any assignment you write should confidently reflect this grade average.

        Here’s how your work aligns with American high school grading standards:
        
        - A (90-100): Excellent
        - B (80-89): Good
        - C (70-79): Satisfactory
        - D (60-69): Limited
        - F (59 and below): Very Limited or Poor

        **Essay Topic:** {prompt}

        Write your essay directly, without repeating the topic instructions. Provide only the content of the essay as if you were submitting it for a grade.
        """

        try:
            # Generate the essay content
            completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a skilled educational content generator."},
                    {"role": "user", "content": essay_prompt}
                ]
            )

            essay = completion.choices[0].message['content'].strip()
            
            # Secondary prompt to generate a confident and clear explanation
            explanation_prompt = f"""
            Based on the following essay, provide a confident, 5-7 sentence ONE paragraph explanation of how this essay fulfills a {quality} grade level for a {grade_level}th-grade {subject} assignment.

            Essay:
            ---
            {essay}
            ---
            Explanation: Be clear and definitive, explaining how specific elements of this essay fulfill a {quality} grade level.
            """
            
            # Generate the confident explanation
            explanation_completion = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a skilled educational content generator providing a clear and confident explanation."},
                    {"role": "user", "content": explanation_prompt}
                ]
            )
            
            explanation = explanation_completion.choices[0].message['content'].strip()
            
            # Append essay and explanation to the list
            essays_with_explanations.append({
                "quality": quality,
                "essay": essay,
                "explanation": explanation
            })
            
        except Exception as e:
            print(f"Error generating essay and explanation for {quality} quality level ({grade_level}th Grade {subject}), Prompt: {prompt}:", e)
            essays_with_explanations.append({
                "quality": quality,
                "essay": None,
                "explanation": None
            })
    
    return essays_with_explanations


In [5]:
# Step 1: Define pairs of grade level and subject
pairs = [(11, "US History")] #(10, "Global History")]



### Calling `generate_prompts`

- Defines `grade_subject_pairs`, a list of grade level and subject combinations for which assignment prompts are needed.
- Calls `generate_prompts` with this list, generating and displaying 10 unique assignment prompts for each grade-subject pair in a readable Markdown format.
- The output, `all_prompts`, is a dictionary that stores prompts by grade-subject pair for further use.




In [6]:
# Step 2: Generate prompts for each grade-subject pair
all_prompts = generate_prompts(pairs)


### Generated Prompts for 11th Grade US History

**Prompt 1:** Write a persuasive essay arguing whether the Civil War was primarily fought over states' rights or slavery, providing evidence to support your position.

**Prompt 2:** Create a timeline tracking the major events leading up to the Great Depression. Include detailed descriptions and analysis of each event's impact on American society.

**Prompt 3:** Analyze the significance of the Montgomery Bus Boycott in the Civil Rights Movement. How did this event contribute to the larger push for desegregation and equal rights in America?

**Prompt 4:** Research and present on the role of women during World War II, focusing on the impact of their contributions to the war effort on the home front and abroad.

...

**Prompt 6:** Investigate the role of Native American tribes in shaping early American history. How did interactions with European settlers impact their way of life and cultural traditions?

**Prompt 7:** Compare and contrast the approaches of Malcolm X and Martin Luther King Jr. to the Civil Rights Movement. How did their differing strategies and ideologies influence the movement?

**Prompt 8:** Examine the impact of the Cold War on American society and politics. How did the fear of communism shape domestic and foreign policies during this period?

**Prompt 9:** Conduct a mock trial reenactment of the Salem Witch Trials. Research the historical context and individuals involved to present a compelling case for or against the accused.

**Prompt 10:** Imagine you are a journalist reporting on the Watergate scandal. Write a detailed news article summarizing the key events leading up to President Nixon's resignation and the aftermath of the scandal on American politics.

In [7]:
# Define possible quality levels
quality_levels = ["A", "B", "C", "D", "F"]

num_essays = 4  # Number of essays to generate per prompt
selected_qualities = {pair: [random.choice(quality_levels) for _ in range(num_essays)] for pair in pairs}

# Display the selected qualities for each pair
print("Selected Quality Levels (Grades) for Each Grade-Subject Pair:")
for pair, qualities in selected_qualities.items():
    grade_level, subject = pair
    print(f"{grade_level}th Grade {subject}: {qualities}")

Selected Quality Levels (Grades) for Each Grade-Subject Pair:
11th Grade US History: ['A', 'C', 'A', 'A']


In [8]:
# Select a random prompt for each grade-subject pair
selected_prompts = [random.choice(all_prompts[pair]) for pair in pairs]

# Display the selected prompts as a formatted list
for i, (pair, prompt) in enumerate(zip(pairs, selected_prompts), start=1):
    grade_level, subject = pair
    print(f"{i}. {grade_level}th Grade {subject} Prompt:")
    print(f"   {prompt}\n")


1. 11th Grade US History Prompt:
   Research and present on the role of women during World War II, focusing on the impact of their contributions to the war effort on the home front and abroad.



### Calling `generate_sample_essay`

- Defines `quality_levels`, a list of possible grades ("A" through "F") to randomly assign to each generated essay.
- Calls `generate_sample_essay` with `all_prompts`, `grade_subject_pairs`, and a specified `prompt_index` (0), generating a sample essay for each grade-subject pair.
- Each essay is displayed with its corresponding quality level and prompt. The output, `all_essays`, is a dictionary that stores the generated essays and their assigned quality levels for further use.







In [9]:
# Step 4: Generate multiple essays with selected qualities for each grade-subject pair
all_essays = {}
for pair in pairs:
    grade_level, subject = pair
    # Select a random prompt for the current grade-subject pair
    prompt = random.choice(all_prompts[pair])
    
    # Get the pre-selected quality levels for this pair
    qualities = selected_qualities[pair]
    
    # Generate multiple essays with the selected qualities for the prompt
    essays_with_varied_qualities = generate_multiple_essays_with_selected_qualities(prompt, grade_level, subject, qualities)
    
    # Store results for the current pair
    all_essays[pair] = {
        "prompt": prompt,
        "essays": essays_with_varied_qualities
    }
    
    # Display the selected prompt
    display(Markdown(f"### Selected Prompt for {grade_level}th Grade {subject}"))
    display(Markdown(f"**Prompt:** {prompt}\n"))
    
    # Display each generated essay with its quality and confident explanation
    for i, essay_data in enumerate(essays_with_varied_qualities, start=1):
        quality = essay_data["quality"]
        essay = essay_data["essay"]
        explanation = essay_data["explanation"]
        
        display(Markdown(f"### Essay {i} (Quality: {quality})"))
        display(Markdown(essay))
        display(Markdown(f"**Explanation:** {explanation}\n"))


### Selected Prompt for 11th Grade US History

**Prompt:** Compare and contrast the approaches of Malcolm X and Martin Luther King Jr. to the Civil Rights Movement. How did their differing strategies and ideologies influence the movement?


### Essay 1 (Quality: A)

Malcolm X and Martin Luther King Jr. were two influential figures during the Civil Rights Movement in the United States, each with their own distinctive approaches and ideologies. While both men aimed to achieve racial equality and justice for African Americans, they followed different paths to reach this common goal.

Martin Luther King Jr., a Baptist minister and a proponent of nonviolent resistance, believed in achieving civil rights through peaceful protest and civil disobedience. He promoted the idea of love and unity among all races and advocated for integration and equality. King emphasized the need for nonviolent direct action to bring about social change, famously leading the Montgomery Bus Boycott and delivering his iconic "I Have a Dream" speech during the March on Washington. His approach was rooted in Christian principles of love, forgiveness, and nonviolence, inspiring widespread support for the Civil Rights Movement.

On the other hand, Malcolm X, a prominent leader in the Nation of Islam, took a more militant and separatist stance in his fight for racial justice. He advocated for self-defense and armed resistance against white supremacy, calling for black empowerment and self-determination. Malcolm X criticized King's nonviolent approach as ineffective and believed in responding to violence with force, if necessary. His message resonated with many African Americans who felt disillusioned with the slow progress of nonviolent tactics, appealing to those seeking more radical solutions to combat racism and oppression.

The differing strategies and ideologies of Malcolm X and Martin Luther King Jr. had a significant impact on the Civil Rights Movement. While King's peaceful resistance strategy gained mainstream support and led to legislative victories such as the Civil Rights Act of 1964 and the Voting Rights Act of 1965, Malcolm X's advocacy for black nationalism and self-defense brought attention to the deep-rooted issues of systemic racism and challenged the status quo. Both men inspired and mobilized millions of people to fight against racial discrimination, albeit through contrasting means.

In conclusion, Malcolm X and Martin Luther King Jr. played pivotal roles in shaping the Civil Rights Movement with their distinct approaches and ideologies. While King's commitment to nonviolent protest and integration helped progress towards racial equality, Malcolm X's militant stance sparked conversations about black empowerment and self-respect. Together, their contributions highlight the complexity and diversity of strategies within the broader fight for civil rights in America.

**Explanation:** This essay demonstrates an A-grade level for an 11th-grade US History assignment by providing a comprehensive and nuanced analysis of Malcolm X and Martin Luther King Jr.'s roles in the Civil Rights Movement. It showcases a deep understanding of the contrasting ideologies and approaches of these two pivotal figures, highlighting King's nonviolent resistance and emphasis on integration, and juxtaposing it with Malcolm X's more militant and separatist stance. The essay effectively discusses how these differing strategies influenced the movement, leading to legislative victories and challenging the status quo. Moreover, it offers a well-structured argument with clear thesis statements, supported by specific examples such as the Montgomery Bus Boycott and Malcolm X's advocacy for black empowerment. Overall, the essay displays critical thinking skills, historical context, and sophisticated analysis, making it deserving of an A grade in an 11th-grade US History assignment.


...

Malcolm X and Martin Luther King Jr. were two prominent leaders in the Civil Rights Movement with contrasting approaches and ideologies, greatly influencing the movement's trajectory.

Martin Luther King Jr., a proponent of nonviolent civil disobedience, believed in the power of love and peaceful resistance to bring about social change. He emphasized integration, equality, and the importance of working alongside white allies to achieve racial justice. King's leadership in organizing nonviolent protests and delivering inspiring speeches, such as his iconic "I Have a Dream" speech, helped galvanize public support for the civil rights cause.

On the other hand, Malcolm X advocated for a more radical approach to racial equality, rejecting nonviolence and promoting self-defense in response to systemic oppression. He was a proponent of Black Nationalism and believed in empowering African Americans through self-reliance and separatism. Malcolm X’s fiery speeches and critiques of white supremacy challenged the status quo and resonated with many disillusioned African Americans seeking a more assertive stance against racial injustices.

These differing strategies and ideologies had a significant impact on the Civil Rights Movement. While King's nonviolent approach attracted broader support from the general public, including white Americans and political leaders, Malcolm X's advocacy for self-defense and Black empowerment appealed to those who felt marginalized and oppressed. Both leaders sparked important conversations about race, equality, and justice in America and inspired many to join the fight for civil rights.

Overall, Martin Luther King Jr. and Malcolm X's contrasting approaches to the Civil Rights Movement underscored the complexity and diversity of strategies within the movement. Their legacies continue to shape discussions on race relations and activism in the United States, reminding us of the ongoing struggle for equality and social justice.

**Explanation:** This essay deserves an A grade for an 11th-grade US History assignment as it demonstrates a comprehensive understanding of the topic by analyzing and comparing the contrasting approaches of Martin Luther King Jr. and Malcolm X in the Civil Rights Movement. It effectively discusses how King's advocacy for nonviolent civil disobedience and integration, alongside Malcolm X's more radical views on self-defense and Black empowerment, influenced the movement's trajectory. The essay showcases a nuanced understanding of the impact of their strategies on attracting different audiences and sparking crucial conversations about race and justice in America. Additionally, it effectively highlights the lasting legacies of both leaders and their contributions to the ongoing struggle for equality, showcasing critical thinking and historical analysis skills expected at an A level.


### Essay 4 (Quality: A)

Malcolm X and Martin Luther King Jr. were two prominent figures in the Civil Rights Movement, each with their own unique approaches and ideologies. While both leaders aimed to achieve racial equality and justice, they had distinct strategies that influenced the movement in significant ways.

Malcolm X was a vocal advocate for Black empowerment and self-defense. His ideology was rooted in black nationalism and he believed in achieving freedom and equality through any means necessary, including self-defense and armed resistance. Malcolm X emphasized the importance of Black pride, self-reliance, and challenging the oppressive white power structure. He rejected nonviolence and criticized the integrationist approach of Martin Luther King Jr., arguing that Black Americans should separate themselves from whites and create their own identity and institutions.

On the other hand, Martin Luther King Jr. was a proponent of nonviolent resistance and civil disobedience. He believed in the power of love, peace, and passive resistance to bring about social change. King emphasized the importance of unity, integration, and working together with white allies to dismantle institutional racism and achieve equality for all. His approach was deeply rooted in Christian principles of forgiveness, love, and nonviolence, and he advocated for peaceful protests, marches, and boycotts to raise awareness and bring about legislative changes.

The differing strategies and ideologies of Malcolm X and Martin Luther King Jr. had a significant impact on the Civil Rights Movement. Malcolm X's bold and confrontational tactics appealed to those who felt frustrated and marginalized by the slow progress of the movement. His emphasis on self-empowerment and self-defense inspired many to take a more assertive stance against racism and discrimination.

In contrast, Martin Luther King Jr.'s commitment to nonviolence and integration resonated with a broader audience, including white supporters and moderate activists. His message of love and unity helped to galvanize a mass movement for civil rights, leading to landmark legislation such as the Civil Rights Act of 1964 and the Voting Rights Act of 1965.

Overall, the approaches of Malcolm X and Martin Luther King Jr. to the Civil Rights Movement were complementary yet divergent. While Malcolm X's militant stance highlighted the urgent need for black liberation and self-determination, King's peaceful activism paved the way for broader social change and integration. Together, their differing strategies and ideologies enriched the Civil Rights Movement and ultimately advanced the cause of racial equality in America.

**Explanation:** This essay demonstrates an A grade level for 11th-grade US History due to its in-depth analysis of the contrasting strategies and ideologies of Malcolm X and Martin Luther King Jr. in the Civil Rights Movement. The writer shows a deep understanding of these two pivotal figures, highlighting their unique approaches towards achieving racial equality. The essay effectively explains Malcolm X's advocacy for black nationalism and self-defense, contrasting it with Martin Luther King Jr.'s belief in nonviolent resistance and integration. Through a well-structured comparison, the essay delves into how these differing tactics impacted the movement and influenced its progress, ultimately leading to significant legislative changes. The writer's clear and comprehensive exploration of these complex historical figures and their impacts on the Civil Rights Movement showcases a high level of critical thinking and historical analysis, earning it an A grade for its depth, clarity, and insightful examination of this crucial period in American history.


In [10]:
explanation

"This essay demonstrates an A grade level for 11th-grade US History due to its in-depth analysis of the contrasting strategies and ideologies of Malcolm X and Martin Luther King Jr. in the Civil Rights Movement. The writer shows a deep understanding of these two pivotal figures, highlighting their unique approaches towards achieving racial equality. The essay effectively explains Malcolm X's advocacy for black nationalism and self-defense, contrasting it with Martin Luther King Jr.'s belief in nonviolent resistance and integration. Through a well-structured comparison, the essay delves into how these differing tactics impacted the movement and influenced its progress, ultimately leading to significant legislative changes. The writer's clear and comprehensive exploration of these complex historical figures and their impacts on the Civil Rights Movement showcases a high level of critical thinking and historical analysis, earning it an A grade for its depth, clarity, and insightful examin

## Rubric 

In [11]:
file_path = '/Users/samuelgartenstein/Desktop/qmss_llm_innovation_lab/Rubrics/General History Rubric.md'
with open(file_path, 'r') as file:
    rubric = file.read()
print(rubric)


# High School History Course Rubric

| Criteria                               | Poor                                                                                                                     | Satisfactory                                                                                                                                            | Good                                                                                                                                                    | Excellent                                                                                                                                               | Total Possible Points |
|----------------------------------------|---------------------------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------

### Rubric Grading

In [29]:
# Function to grade an essay based on a rubric and directly extract the letter grade from the response
def grade_essay_with_rubric(essay, rubric, grade_level, subject):
    primary_prompt = f"""
    You are an experienced grader for a high school {subject} course. Grade the essay according to the rubric provided.

    {rubric}

    Here is the essay to be graded:
    ---
    {essay}
    ---

    For each criterion in the rubric:
    1. Assign a score out of 25 points in the format "Score: X/25".
    2. Provide a 2-3 sentence explanation of the score, referencing specific aspects of the essay.

    After evaluating each criterion, at the end, please include:
    - **Total Score**: X/100
    - **Letter Grade**: Determine the letter grade based on this scale:
      - A: 90-100
      - B: 80-89
      - C: 70-79
      - D: 60-69
      - F: 59 and below

    Ensure that the Total Score and Letter Grade appear clearly at the very end of the response.
    """

    # Prompt for a summary explanation
    summary_prompt_template = """
    Based on the following grading breakdown, provide a one-paragraph explanation summarizing the main reasons for the overall grade based on the criteria:
    ---
    {grading_breakdown}
    ---
    Ensure the explanation captures the overall strengths and weaknesses without referencing specific scores.
    """

    try:
        # Get grading breakdown and initial rubric-based grade
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an experienced grader familiar with high school rubrics."},
                {"role": "user", "content": primary_prompt}
            ]
        )
        grading_response = completion.choices[0].message['content'].strip()

        # Try to directly extract the letter grade from the response
        letter_grade_match = re.search(r"Letter Grade:\s*([A-F])", grading_response)
        letter_grade = letter_grade_match.group(1) if letter_grade_match else "Grade not available"

        # Extract the total score (for reference or further analysis if needed)
        score_match = re.search(r"Total Score:\s*(\d+)", grading_response)
        total_score = int(score_match.group(1)) if score_match else None

        # Generate rubric-based explanation
        summary_prompt = summary_prompt_template.format(grading_breakdown=grading_response)
        explanation_completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an experienced grader summarizing the grading breakdown."},
                {"role": "user", "content": summary_prompt}
            ]
        )
        explanation_only = explanation_completion.choices[0].message['content'].strip()

        return grading_response.strip(), explanation_only, letter_grade

    except Exception as e:
        print(f"Error grading essay: {e}")
        return "Grading content not available", "Explanation not available", "Grade not available"

# Example usage for grading multiple essays for each grade-subject pair
for pair in pairs:
    grade_level, subject = pair
    essays = all_essays[pair]["essays"]  # Assuming all_essays[pair]["essays"] contains a list of essays for each pair

    # Grade each essay in the list
    for idx, essay_data in enumerate(essays, start=1):
        essay_text = essay_data["essay"]

        # Get the grading breakdown, explanation, and letter grade for each essay
        grading_content, explanation, letter_grade = grade_essay_with_rubric(essay_text, rubric, grade_level, subject)

        # Display results in Markdown format
        display(Markdown(f"### Essay Grading Results for {grade_level}th Grade {subject}"))
        display(Markdown(f"**Essay Content:**"))
        display(Markdown(essay_text))
        display(Markdown(f"**Grading Breakdown:**"))
        display(Markdown(grading_content))
        display(Markdown(f"**Explanation:** {explanation}"))
        display(Markdown(f"**Letter Grade:** {letter_grade}\n"))  # Display letter grade at the bottom


### Essay Grading Results for 11th Grade US History

**Essay Content:**

Malcolm X and Martin Luther King Jr. were two influential figures during the Civil Rights Movement in the United States, each with their own distinctive approaches and ideologies. While both men aimed to achieve racial equality and justice for African Americans, they followed different paths to reach this common goal.

Martin Luther King Jr., a Baptist minister and a proponent of nonviolent resistance, believed in achieving civil rights through peaceful protest and civil disobedience. He promoted the idea of love and unity among all races and advocated for integration and equality. King emphasized the need for nonviolent direct action to bring about social change, famously leading the Montgomery Bus Boycott and delivering his iconic "I Have a Dream" speech during the March on Washington. His approach was rooted in Christian principles of love, forgiveness, and nonviolence, inspiring widespread support for the Civil Rights Movement.

On the other hand, Malcolm X, a prominent leader in the Nation of Islam, took a more militant and separatist stance in his fight for racial justice. He advocated for self-defense and armed resistance against white supremacy, calling for black empowerment and self-determination. Malcolm X criticized King's nonviolent approach as ineffective and believed in responding to violence with force, if necessary. His message resonated with many African Americans who felt disillusioned with the slow progress of nonviolent tactics, appealing to those seeking more radical solutions to combat racism and oppression.

The differing strategies and ideologies of Malcolm X and Martin Luther King Jr. had a significant impact on the Civil Rights Movement. While King's peaceful resistance strategy gained mainstream support and led to legislative victories such as the Civil Rights Act of 1964 and the Voting Rights Act of 1965, Malcolm X's advocacy for black nationalism and self-defense brought attention to the deep-rooted issues of systemic racism and challenged the status quo. Both men inspired and mobilized millions of people to fight against racial discrimination, albeit through contrasting means.

In conclusion, Malcolm X and Martin Luther King Jr. played pivotal roles in shaping the Civil Rights Movement with their distinct approaches and ideologies. While King's commitment to nonviolent protest and integration helped progress towards racial equality, Malcolm X's militant stance sparked conversations about black empowerment and self-respect. Together, their contributions highlight the complexity and diversity of strategies within the broader fight for civil rights in America.

**Grading Breakdown:**

**1. Evidence and Analysis**
- **Score: 23/25**
- The essay provides specific and relevant evidence about the different approaches of Malcolm X and Martin Luther King Jr. during the Civil Rights Movement. It thoroughly analyzes their strategies, connects ideas to the historical context, and explains the impact of their ideologies on the movement.

**2. Understanding Key Ideas**
- **Score: 24/25**
- The essay demonstrates a clear understanding of the main ideas presented, accurately summarizing the distinct approaches of Malcolm X and Martin Luther King Jr. It highlights key details and effectively conveys the relationships between their ideologies and the broader Civil Rights Movement.

**3. Cause and Effect Reasoning**
- **Score: 22/25**
- The essay shows a good analysis of the cause-and-effect relationships between the strategies of Malcolm X and Martin Luther King Jr. Although insightful at times, the reasoning could be further developed to provide a deeper exploration of how their actions influenced the outcomes of the movement.

**4. Writing Clarity and Organization**
- **Score: 23/25**
- The writing is clear, coherent, and well-organized, presenting the historical information in a logical and compelling narrative. While the essay maintains a good structure, there are minor instances where clarity could be improved for a more seamless flow of ideas.

**Total Score: 92/100**
- **Letter Grade: A**

Overall, the essay effectively analyzes the strategies of Malcolm X and Martin Luther King Jr. within the context of the Civil Rights Movement, providing a well-rounded discussion of their impact and significance. It presents a thorough understanding of the historical topic with clear writing and organization, earning an "A" grade.

...

Malcolm X and Martin Luther King Jr. were two prominent figures in the Civil Rights Movement, each with their own unique approaches and ideologies. While both leaders aimed to achieve racial equality and justice, they had distinct strategies that influenced the movement in significant ways.

Malcolm X was a vocal advocate for Black empowerment and self-defense. His ideology was rooted in black nationalism and he believed in achieving freedom and equality through any means necessary, including self-defense and armed resistance. Malcolm X emphasized the importance of Black pride, self-reliance, and challenging the oppressive white power structure. He rejected nonviolence and criticized the integrationist approach of Martin Luther King Jr., arguing that Black Americans should separate themselves from whites and create their own identity and institutions.

On the other hand, Martin Luther King Jr. was a proponent of nonviolent resistance and civil disobedience. He believed in the power of love, peace, and passive resistance to bring about social change. King emphasized the importance of unity, integration, and working together with white allies to dismantle institutional racism and achieve equality for all. His approach was deeply rooted in Christian principles of forgiveness, love, and nonviolence, and he advocated for peaceful protests, marches, and boycotts to raise awareness and bring about legislative changes.

The differing strategies and ideologies of Malcolm X and Martin Luther King Jr. had a significant impact on the Civil Rights Movement. Malcolm X's bold and confrontational tactics appealed to those who felt frustrated and marginalized by the slow progress of the movement. His emphasis on self-empowerment and self-defense inspired many to take a more assertive stance against racism and discrimination.

In contrast, Martin Luther King Jr.'s commitment to nonviolence and integration resonated with a broader audience, including white supporters and moderate activists. His message of love and unity helped to galvanize a mass movement for civil rights, leading to landmark legislation such as the Civil Rights Act of 1964 and the Voting Rights Act of 1965.

Overall, the approaches of Malcolm X and Martin Luther King Jr. to the Civil Rights Movement were complementary yet divergent. While Malcolm X's militant stance highlighted the urgent need for black liberation and self-determination, King's peaceful activism paved the way for broader social change and integration. Together, their differing strategies and ideologies enriched the Civil Rights Movement and ultimately advanced the cause of racial equality in America.

**Grading Breakdown:**

**1. Evidence and Analysis**
- **Score: 23/25**
- **Explanation:** The essay provides specific and relevant evidence about Malcolm X and Martin Luther King Jr.'s strategies and ideologies. Thorough analysis is evident throughout the essay, connecting ideas to the historical topic of the Civil Rights Movement effectively.

**2. Understanding Key Ideas**
- **Score: 22/25**
- **Explanation:** The essay accurately identifies and summarizes the main ideas of Malcolm X and Martin Luther King Jr., although there could have been a bit more clarity in highlighting the relationships among key details and ideas.

**3. Cause and Effect Reasoning**
- **Score: 24/25**
- **Explanation:** The essay insightfully analyzes the cause-and-effect relationships between the strategies and ideologies of Malcolm X and Martin Luther King Jr. The reasoning is well-developed and helps in understanding the impact of their differing approaches on the Civil Rights Movement.

**4. Writing Clarity and Organization**
- **Score: 24/25**
- **Explanation:** The writing is clear, coherent, and well-organized, presenting a logical and compelling narrative about the distinct strategies of Malcolm X and Martin Luther King Jr. The essay maintains a good flow and structure.

**Total Score: 93/100**
**Letter Grade: A**

**Explanation:** The student's essay on Malcolm X and Martin Luther King Jr. excels in providing specific evidence and in-depth analysis of their strategies and ideologies within the context of the Civil Rights Movement, showcasing a strong understanding of the historical topic. While the essay effectively demonstrates cause-and-effect reasoning in analyzing the impact of the leaders' approaches, there could have been a slight improvement in clearly highlighting the relationships among key ideas. Additionally, the writing is highly commendable for its clarity, coherence, and strong organizational structure, contributing to a logical and engaging narrative. Overall, the essay's strengths lie in its detailed analysis, insightful reasoning, and well-crafted narrative, resulting in an impressive grade of A.

**Letter Grade:** A


## Comparison

In [31]:
# Example usage for grading and comparison
for pair in pairs:
    grade_level, subject = pair
    essays = all_essays[pair]["essays"]  # Assuming all_essays[pair]["essays"] contains a list of essays for each pair

    for idx, essay_data in enumerate(essays, start=1):
        essay_text = essay_data["essay"]  # Access the actual essay text
        llm_grade = essay_data["quality"]   # Retrieve the LLM-generated grade (A, B, C, etc.)

        # Use the grading function to get the grading breakdown, rubric explanation, and letter grade
        grading_content, rubric_explanation, letter_grade = grade_essay_with_rubric(essay_text, rubric, grade_level, subject)
        
        # Access the LLM-generated explanation
        llm_explanation = essay_data["explanation"]

        # Display the essay content, explanations, and compare grades
        display(Markdown(f"### Comparison for Essay {idx} (LLM Grade: {llm_grade}) - {grade_level}th Grade {subject}"))
        display(Markdown(f"**Essay Content:**\n\n{essay_text}"))
        display(Markdown(f"**LLM Explanation:**\n\n{llm_explanation}"))
        display(Markdown(f"**Rubric Explanation:**\n\n{rubric_explanation}"))
        display(Markdown(f"**LLM Grade:** {llm_grade}"))
        display(Markdown(f"**Letter Grade:** {letter_grade}"))
        display(Markdown("---"))



### Comparison for Essay 1 (LLM Grade: A) - 11th Grade US History

**Essay Content:**

Malcolm X and Martin Luther King Jr. were two influential figures during the Civil Rights Movement in the United States, each with their own distinctive approaches and ideologies. While both men aimed to achieve racial equality and justice for African Americans, they followed different paths to reach this common goal.

Martin Luther King Jr., a Baptist minister and a proponent of nonviolent resistance, believed in achieving civil rights through peaceful protest and civil disobedience. He promoted the idea of love and unity among all races and advocated for integration and equality. King emphasized the need for nonviolent direct action to bring about social change, famously leading the Montgomery Bus Boycott and delivering his iconic "I Have a Dream" speech during the March on Washington. His approach was rooted in Christian principles of love, forgiveness, and nonviolence, inspiring widespread support for the Civil Rights Movement.

On the other hand, Malcolm X, a prominent leader in the Nation of Islam, took a more militant and separatist stance in his fight for racial justice. He advocated for self-defense and armed resistance against white supremacy, calling for black empowerment and self-determination. Malcolm X criticized King's nonviolent approach as ineffective and believed in responding to violence with force, if necessary. His message resonated with many African Americans who felt disillusioned with the slow progress of nonviolent tactics, appealing to those seeking more radical solutions to combat racism and oppression.

The differing strategies and ideologies of Malcolm X and Martin Luther King Jr. had a significant impact on the Civil Rights Movement. While King's peaceful resistance strategy gained mainstream support and led to legislative victories such as the Civil Rights Act of 1964 and the Voting Rights Act of 1965, Malcolm X's advocacy for black nationalism and self-defense brought attention to the deep-rooted issues of systemic racism and challenged the status quo. Both men inspired and mobilized millions of people to fight against racial discrimination, albeit through contrasting means.

In conclusion, Malcolm X and Martin Luther King Jr. played pivotal roles in shaping the Civil Rights Movement with their distinct approaches and ideologies. While King's commitment to nonviolent protest and integration helped progress towards racial equality, Malcolm X's militant stance sparked conversations about black empowerment and self-respect. Together, their contributions highlight the complexity and diversity of strategies within the broader fight for civil rights in America.

**LLM Explanation:**

This essay demonstrates an A-grade level for an 11th-grade US History assignment by providing a comprehensive and nuanced analysis of Malcolm X and Martin Luther King Jr.'s roles in the Civil Rights Movement. It showcases a deep understanding of the contrasting ideologies and approaches of these two pivotal figures, highlighting King's nonviolent resistance and emphasis on integration, and juxtaposing it with Malcolm X's more militant and separatist stance. The essay effectively discusses how these differing strategies influenced the movement, leading to legislative victories and challenging the status quo. Moreover, it offers a well-structured argument with clear thesis statements, supported by specific examples such as the Montgomery Bus Boycott and Malcolm X's advocacy for black empowerment. Overall, the essay displays critical thinking skills, historical context, and sophisticated analysis, making it deserving of an A grade in an 11th-grade US History assignment.

**Rubric Explanation:**

The essay on the contrasting approaches of Malcolm X and Martin Luther King Jr. in the Civil Rights Movement earns a "B" grade due to its solid performance in most criteria. It excels in providing relevant evidence and analysis, accurately summarizing key ideas, and maintaining clear writing organization and coherence. The essay effectively connects the ideologies of both figures to the historical context and demonstrates a clear understanding of their roles in the movement. While there is room for improvement in providing deeper analysis and more detailed cause-and-effect reasoning, a slightly stronger coherence could further enhance the overall clarity of the essay. Overall, the essay displays a commendable grasp of the subject matter with a few areas that could be strengthened for an even more comprehensive evaluation.

**LLM Grade:** A

...

**LLM Explanation:**

This essay demonstrates an A grade level for 11th-grade US History due to its in-depth analysis of the contrasting strategies and ideologies of Malcolm X and Martin Luther King Jr. in the Civil Rights Movement. The writer shows a deep understanding of these two pivotal figures, highlighting their unique approaches towards achieving racial equality. The essay effectively explains Malcolm X's advocacy for black nationalism and self-defense, contrasting it with Martin Luther King Jr.'s belief in nonviolent resistance and integration. Through a well-structured comparison, the essay delves into how these differing tactics impacted the movement and influenced its progress, ultimately leading to significant legislative changes. The writer's clear and comprehensive exploration of these complex historical figures and their impacts on the Civil Rights Movement showcases a high level of critical thinking and historical analysis, earning it an A grade for its depth, clarity, and insightful examination of this crucial period in American history.

**Rubric Explanation:**

The essay on the comparison between Malcolm X and Martin Luther King Jr. earned an 'A' grade due to its strong performance across all criteria. It excelled in providing specific and relevant evidence, thorough analysis connecting the leaders' strategies and impact, and demonstrating a clear understanding of their ideologies. The cause-and-effect reasoning was effective, though with potential for deeper insight in some areas. Moreover, the writing displayed clarity, coherence, and logical organization, making the comparison compelling and easy to follow. Overall, the essay exhibited a high level of proficiency in analyzing the differences between Malcolm X and Martin Luther King Jr., thereby earning a top grade.

**LLM Grade:** A

**Letter Grade:** A

---